In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

### 0. 선행 준비

In [ ]:
import os
import sys
from openai import OpenAI
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError
from tabulate import tabulate
import json
from json.decoder import JSONDecodeError

client = OpenAI()

engine = create_engine('sqlite:///user_support.db', echo=False)

def init_meta_data():
    # 메타데이터 작성, 테이블 정보를 반영하기 위해 엔진에서 테이블 정보를 읽음
    metadata = MetaData()
    metadata.reflect(bind=engine)

    # 메타데이터의 테이블 정보로부터 테이블 이름과 컬럼 이름을 편집
    meta_text = ''
    for table in metadata.tables.values():
        meta_text += '테이블 이름: '+ table.name + '\n'
        for column in table.columns:
            meta_text += f'컬럼 이름: {column.name}, 형식: {column.type}\n'
        meta_text += '\n'
    return meta_text

def exec_sql(sql):
    Session = sessionmaker(bind=engine)

    error_text = "None"
    rows = []
    tabled = ""
    dict_array = []

    try:
        with Session() as session:
            t = text (sql)
            result = session.execute(t)

            header = [k for k in result.keys()]

            rows = result.fetchall()
            tabled = tabulate(rows,header, tablefmt="plain")

            dict_array = []
            for row in rows:
                dict_row = {}
                for i in range(len(header)):
                    dict_row[header[i]] = row[i]
                dict_array.append(dict_row)
            #print(tabled)
            #print()

    except SQLAlchemyError as e:
        print(f'Exception Excute SQL: {e}\n')
        error_text = str(e)

    return {
        "read_count": len(rows),
        "tabled": tabled,
        "dict_array": dict_array,
        "error": error_text
    }

def exec_api(user_text="", user_info="", prompt="", meta_text="", summary="", temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = temperature,
        messages=[
            {
                "role": "user",
                        "content": prompt.format(
                            user_text=user_text,
                            user_info=user_info,
                            prompt=prompt,
                            meta_text=meta_text,
                            summary=summary)
            }
        ],
    )

    return response.choices[0].message.content.strip()

### 1. 사용자 식별 작업

In [ ]:
def find_user(user_text):
    prompt="""
        {{
            "prompt": "
            ## DB 정보
            {meta_text}

            ## 처리
            사용자 테이블을 참조하여 다음 요청문에 대해 고유한 사용자를 식별하고 싶습니다.
            그 답변(SQL문은 포함하지 않음)과 SQL문(ID, 이름, 전화번호, 이메일)을 엄격한 JSON 형식으로 다음과 같이 제공해주시기 바랍니다.
            참고로, 한글 이름은 last_name이 먼저입니다.
            설명문은 필요하지 않으므로 표시하지 않도록 해주세요.:
                {{
                    "response": {{
                        "message": "사용자를 DB에 질의합니다",
                        "sql_query": "SQL 문"
                    }}
                }}

                요청: [{user_text}]
            }}
        }}
    """

    meta_text = init_meta_data()
    response = exec_api(user_text=user_text, prompt=prompt, meta_text=meta_text)
    # print(response)
    json_answer = json.loads(response)
    sql = json_answer['response']['sql_query']

    if sql:
        result = exec_sql(sql)
        # print(result['read_count'])
        # print(result['tabled'])
        # print(result['dict_array'])
        if result['read_count'] == 1:
            return result['tabled']
        else:
            return ""

if __name__ == "__main__":
    print("고객 정보를 확인합니다. 이름, 전화번호, 사용자 ID, 이메일 등 고객을 식별할 수 있는 정보를 입력해주세요.")

    while True:
        user_text = input("\n>")
        user = find_user(user_text)
        print(user)

        if user:
            print("고객 정보를 확인했습니다. 감사합니다.")
            break
        else:
            print("고객 정보를 확인할 수 없습니다. 다시 입력해주세요.")

### 2. 기본 정보 획득 작업

In [ ]:
def get_user_info(user):
    prompt = """
    {{
        "prompt": "
        ## DB 정보
        {meta_text}

        ## 사용자 정보
        {user_info}

        ## 처리
        DB 정보를 참조하여,
        사용자 정보를 바탕으로 해당 사용자에 대한 모든 것을 알 수 있는 작업
        (구매 이력, 대응 이력, 구매 완료 제품 정보 등)을 나열하고,
        해당 작업의 설명(SQL 문장은 포함하지 않음)과 제목, SQL 문장을
        엄격한 JSON 형식으로 다음과 같이 제공해주세요.
        설명문은 불필요하므로 숨겨주시기 바랍니다.:
            {{
                "message":"(답변 메시지)",
                "queries": [{{
                    "title": "(작업 제목)",
                    "description": "(작업 설명)",
                    "sql_query": "(SQL 문)"
                }}]
            }}
    }}
    """

    meta_text = init_meta_data()
    user_text = ""
    response = exec_api(user_info=user, prompt=prompt, meta_text=meta_text)
    # print(response)
    json_answer = json.loads(response)
    user_info = "\n사용자 정보\n" + user + "\n"

    for query in json_answer['queries']:
        description = query['description']
        user_info += "\n" + query['title'] + "\n"
        # print(title)
        sql = query['sql_query']

        if sql:
            result = exec_sql(sql)
            user_info +=  result['tabled']+"\n"
    return user_info

if __name__ == "__main__":
    print("고객님의 정보를 데이터베이스에 질의하고 있습니다. 잠시 기다려 주십시오.")
    user_info = get_user_info(user)
    print(user_info)
    print("고객님의 정보를 데이터베이스에서 읽어왔습니다.")

### 3. 사용자 지원 작업

In [ ]:
def user_support(user_info):
    summary = ""
    prompt = """
        {{
            "prompt": "
            ## 대화 요약
            {summary}
            ## 사용자 정보
            {user_info}

            ## 처리
            당신은 고객 응대 전문가입니다. 고객의 요청문에 대해 정확하게 답변해주세요.
            고객 응대 시, 사용자 정보를 참고하여 친절하고 정중하게 고객 응대를 해주세요.
            단, DB 정보에 있는 사실만 답변해주세요.
            고객의 요청글이 "감사합니다. 그럼 또 뵙겠습니다. 안녕히 가세요." 등 종료를 암시하는 문구가 입력된 경우 지원 종료로 간주합니다.
            답변을 포함하여 다음과 같이 엄격한 JSON 형식으로 출력해주시기 바랍니다.:
                {{
                    "message": "(답변 메시지)",
                    "status": "(지원 상태. 지원 시작, 지원 중, 지원 종료)",
                    "summary": "(요청문과 답변 내용 요약)"
                }}

            요청문[{user_text}]
            "
        }}
    """

    while True:
        user_text = input(">")
        response = exec_api(user_text=user_text, user_info=user_info, prompt=prompt, summary=summary, temperature=0.5)
        # print(response)
        json_answer = json.loads(response)
        message = json_answer['message']
        print(f"{message}\n")

        if json_answer['status'] == '지원 종료':
            return summary
        summary += f"요청: {user_text}\n답변: {json_answer['message']}\n"

if __name__ == "__main__":
     print("고객님, 궁금한 사항이나 불편한 점이 있으십니까?")
     meta_text = init_meta_data()
     result_summary = user_support(user_info)
     # print(result_summary)
     print("감사합니다. 그럼 이만 실례하겠습니다.")